# Visualization of detected peak of CO2

In [1]:
%matplotlib inline

In [ ]:
#!conda install geopandas
#!pip install geopandas pandas_bokeh

In [ ]:
import os
import glob
import numpy as np
import pandas as pd
# Import geopandas package
import geopandas as gpd
import matplotlib
import matplotlib.pyplot as plt
from mpl_toolkits.basemap import Basemap
from matplotlib.pyplot import figure
from tqdm import tqdm_notebook as tqdm
from shapely.geometry import Point # Shapely for converting latitude/longtitude to geometry

input_dir = r'../../../datasets/OCO2/csv/'
#input_dir = r'../pipeline/'
# peak_dataoco2_1808-o21714-i37483

In [ ]:
dir = os.path.realpath(input_dir)
print(dir)
csv_list = glob.glob(input_dir + "result_"+"*.csv")
# # Initialize array to store data
df_res = pd.DataFrame()
# Loop over the files
for one_file in tqdm(csv_list):
    #print('Reading', one_file)
    df_temp = pd.read_csv(one_file, sep=",")
    df_res = df_res.append(df_temp)



# creating a geometry column 
geometry = [Point(xy) for xy in zip(df_res['longitude'], df_res['latitude'])]

# Coordinate reference system : WGS84
crs = {'init': 'epsg:4326'}

# Creating a Geographic data frame 
gdf_res = gpd.GeoDataFrame(df_res.copy(), crs=crs, geometry=geometry)



In [ ]:
df_res.tail(3)

In [ ]:
gdf_res.tail(3)

In [ ]:
len(df_res)

In [ ]:
# Plot all points
gdf_res.plot(marker='o', color='b', markersize=0.5)

In [ ]:
def draw_map_df(data, x="longitude", y="latitude", c="xco2", lon_min=-180, lon_max=180, lat_min=-90, lat_max=90, size_point=1, frontier=False):

    plt.figure(figsize=(15, 10), edgecolor='w')
    m = Basemap(llcrnrlat=lat_min, urcrnrlat=lat_max, llcrnrlon=lon_min, urcrnrlon=lon_max)
    
    m.shadedrelief()
    
    parallels = np.arange(-80.,81,10.)
    m.drawparallels(parallels,labels=[False,True,True,False])

    meridians = np.arange(10.,351.,20.)
    m.drawmeridians(meridians,labels=[True,False,False,True])

    normal = matplotlib.colors.LogNorm(vmin=data[c].min(), vmax=data[c].max())

    m.scatter(data[x], data[y], c=data[c], cmap=plt.cm.jet, s=size_point, norm=normal)

    if (frontier):
      m.drawcountries(linewidth=0.5)
      m.drawcoastlines(linewidth=0.7)

    plt.show()

In [ ]:
figure(num=None, figsize=(10, 8), dpi=80, facecolor='w', edgecolor='k')
draw_map_df(df_res, c='delta', lon_min=min(df_res['longitude']), lon_max=max(df_res['longitude']), lat_min=min(df_res['latitude']), lat_max=max(df_res['latitude']))

In [ ]:
#!conda install bokeh

## Conversion de coordonnées

On peut se référer aux projections à travers les codes « EPSG », qui sont des identificateurs de SIG nommés et gérés par l’European Petroleum Survey Group.
La projection Sphérique de Mercator est identifiée comme EPSG: 3857. Ces coordonnées sont en mètres et exprimées par x / y.

L’autre code EPSG commun est EPSG:4326, qui utilise le WGS84 comme système de coordonnées. Il décrit des coordonnées (latitude / longitude).

Les données OCO2 sont en EPSG:4326 alors que Bokeh attend des coordonnées EPSG:3857. Il faut donc faire une conversion.

In [ ]:
import pyproj

gps_projection = pyproj.Proj("EPSG:4326")  # GPS
mercator_projection = pyproj.Proj("EPSG:3857")  # Mercator

df_res['x'], df_res['y'] = pyproj.transform(gps_projection, mercator_projection, df_res['latitude'].tolist(), df_res['longitude'].tolist())
df_res['size'] = df_res['delta']*4
df_res.head(3)

In [ ]:
df_res.tail(3)

In [ ]:
# Crash notebook :-(
# import pandas_bokeh
# pandas_bokeh.output_notebook()
# df_res.plot_bokeh(simplify_shapes=10000)

In [ ]:
df_res.columns

In [ ]:
from bokeh.plotting import figure, output_file, show
#from bokeh.tile_providers import CARTODBPOSITRON, get_provider
from bokeh.tile_providers import CARTODBPOSITRON, get_provider
from bokeh.models import ColumnDataSource, HoverTool
from bokeh.models import GeoJSONDataSource
from bokeh.io import output_notebook
# from bokeh.events import ButtonClick
# from bokeh.models import Button, CustomJS
# button = Button()
# button.js_on_event(ButtonClick, CustomJS(code='console.log("JS:Click")'))

output_notebook()

source = ColumnDataSource(data=df_res)
output_file("peak_map.html")

tile_provider = get_provider(CARTODBPOSITRON)
hover = HoverTool(tooltips=[
    ("Orbit", "@orbit"),
    ("sounding_id", "@sounding_id"),
    ("delta", "@delta"),
    ("windspeed_u","@windspeed_u")
])

p = figure(x_range=(-3000000, 5000000), y_range=(-100000, 8000000), plot_width=1200, plot_height=800,
           x_axis_type="mercator", y_axis_type="mercator"
          ,tools=[hover, 'wheel_zoom','save','box_zoom','reset']
          )
p.add_tile(tile_provider)

p.circle(x='x', y='y', source=source,size='size',
         line_color="#FF0000", 
         fill_color="#FF0000",
         fill_alpha=0.01)

show(p)

In [ ]:
# ODIAC - https://colab.research.google.com/drive/1pERH_iSkIkMYy1MDe_MAWTUs2MoPRa7x#scrollTo=uR8cnThg-paEwc
# path6 = "/content/drive/My Drive/Data For Good/odiac2019_1x1d_2018.nc"
# http://datasets.wri.org/dataset/globalpowerplantdatabase%22

In [ ]:
# from bokeh.plotting import figure, show
# from bokeh.io import output_notebook
# output_notebook()
# p = figure(plot_width=400, plot_height=400)
# p.circle([1, 2, 3, 4, 5], [6, 7, 2, 4, 5], size=15, line_color="navy", fill_color="orange", fill_alpha=0.5)
# show(p)

## View one peak

In [ ]:
import os
import glob
import numpy as np
import pandas as pd

def load_one_peak_data(sounding_id, df_all_peak):
    json_list = glob.glob(input_dir + "peak_*-si_*"+sounding_id+"*.json")
    if len(json_list) > 1:
        print("E : more than one result !")
        return False
    if len(json_list) < 1:
        print("ERROR : no result !")
        return False
    one_file = json_list[0]
    print(one_file)
    df_peak = pd.read_json(one_file)
    df_param = df_all_peak.query("sounding_id==2018080404560303")
    param_index = df_param.index[0]
    gaussian_param = {
        'slope' : df_param.loc[param_index, 'slope'],
        'intercept' : df_param.loc[param_index, 'intercept'],
        'amplitude' : df_param.loc[param_index, 'amplitude'],
        'sigma': df_param.loc[param_index, 'sigma'],
        'delta': df_param.loc[param_index, 'delta'],
        'R' : df_param.loc[param_index, 'R'],
    }
    return df_peak, gaussian_param

'''
x : the data input value
m : the slope of the data
b : the intercep of the data
A : Amplitude de la courbe
sig : sigma / écart type de la courbe
'''
def gaussian(x, m, b, A, sig):
    return m * x + b + A / (sig * (2 * np.pi)**0.5) * np.exp(-x**2 / (2*sig**2))

def plot_peak(df_peak, gaussian_param):
    x = df_peak['distance']
    y = df_peak['xco2']
    y = y - gaussian_param['slope'] * x - gaussian_param['intercept']
    plt.scatter(x, y, c=y, s=3, label='sounding')
    plt.plot(x, gaussian(x, m=0, b=0, A=gaussian_param['amplitude'], sig=gaussian_param['sigma'])-gaussian_param['delta'], 'r', label='fit')
    plt.legend()
    plt.title('OCO 2 data')
    plt.xlabel('Distance')
    plt.ylabel('CO²')
    plt.show()

In [ ]:
df_peak, gaussian_param = load_one_peak_data("2018082505142233", df_res)
print(gaussian_param)
# df_res.query("sounding_id==2018080404560303")
# #df_peak.query("distance>-0.1 and distance<0.1")
# df_peak.query("distance == 0.0")
plot_peak(df_peak, gaussian_param)
# Best m, b, A, sig =  [3.92560177e-03 4.00770662e+02 1.43374694e+02 1.42527377e+01]

In [ ]:
df_peak

In [ ]:
from scipy.optimize import curve_fit
import numpy as np

def peak_detection(df_slice):
    default_return = {}
    orbit_index=0
    # Skip if too few data
    if len(df_slice)<400:
        #print('ERROR : Not enought data')
        return default_return
    med_temp = np.median(df_slice['xco2'])
    # std_temp = np.std(df_slice['xco2']) # Not used
    df_slice['xco2_enhancement'] = df_slice['xco2'] - med_temp
    # Base parameters for : m, b, A, sig
    p0 = (0.,med_temp,30*df_slice.loc[0,'xco2_enhancement'],10.) 
    #print('Estimated parameters:', p0)
    d_centered = df_slice
    '''
    Gaussian Fit
    scipy.optimize.curve_fit
    scipy.optimize.curve_fit(f, xdata, ydata, p0=None, sigma=None, absolute_sigma=default_return, check_finite=True, bounds=(-inf, inf), method=None, jac=None, **kwargs)[source]¶
    p0 = Initial guess for the parameters (length N).
    sigma : Determines the uncertainty in ydata.
    '''
    popt, _ = curve_fit(f=gaussian, xdata=d_centered, ydata=df_slice['xco2'], sigma = df_slice['xco2_uncert'], p0 = p0, maxfev=20000, ftol=0.5, xtol=0.5) # ftol=0.5, xtol=0.5 to speed up
    sig = abs(popt[3])  # sigma of the Gaussian (km)
    #print(sig)
    if sig < 2 : return default_return  # too narrow
    if 3*sig > window / 2.: return default_return  # too large
    delta = popt[2]/(popt[3]*(2 * np.pi)**0.5)  # height of the peak (ppm)
    if delta < 0: return default_return  # depletion
    #d_plume = df_slice[(d_centered >= -2*sig) & (d_centered <= 2*sig)]
    #d_backg = df_slice[(d_centered < -2*sig) | (d_centered > 2*sig)]

    # we want at least 1 1-km-sounding per km on average on both sides of the peak within 2 sigmas and between 2 and 3 sigmas
    if len(df_slice[(d_centered >= -1*sig) & (d_centered <= 0)]) < int(sig): return default_return
    if len(df_slice[(d_centered <= 1*sig) & (d_centered >= 0)]) < int(sig): return default_return
    if len(df_slice[(d_centered >= -3*sig) & (d_centered <= -2*sig)]) < int(sig): return default_return
    if len(df_slice[(d_centered <= 3*sig) & (d_centered >= 2*sig)]) < int(sig): return default_return
    # check the quality of the fit
    d_peak = df_slice[(d_centered >= -4*sig) & (d_centered <= 4*sig)]
    d_peak_distance = d_peak['distance'] - df_slice.loc[0, 'distance']
    R = np.corrcoef(gaussian(d_peak_distance,*popt), d_peak['xco2'])
    if R[0,1]**2 < 0.25 : return default_return
    #print('orbit_index',orbit_index, 'Number of good fit',good_find, 'Sigma:', sig, 'Ampleur de l\'émission de CO²:',delta,'Coef de coreflation',R[0,1])
    # TODO: Add filename of input to be able to load it later
    peak = {
        'sounding_id' : df_slice.loc[orbit_index, 'sounding_id'],
        'latitude' : df_slice.loc[orbit_index, 'latitude'],
        'longitude' : df_slice.loc[orbit_index, 'longitude'],
        'orbit' : orbit_number,
        'slope' : popt[0],
        'intercept' : popt[1],
        'amplitude' : popt[2],
        'sigma': popt[3],
        'delta': delta,
        'R' : R[0,1],
        'windspeed_u' : df_slice.loc[0, 'windspeed_u'],
        'windspeed_v' : df_slice.loc[orbit_index, 'windspeed_v']
    }
    return peak


In [ ]:
peak_detection(df_peak)

In [ ]:
df_peak['xco2']